In [1]:
!wget http://cs.stanford.edu/people/karpathy/char-rnn/linux_input.txt

--2017-07-05 08:57:28--  http://cs.stanford.edu/people/karpathy/char-rnn/linux_input.txt
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6206996 (5.9M) [text/plain]
Saving to: ‘linux_input.txt’

linux_input.txt     100%[===================>]   5.92M   447KB/s    in 13s     

2017-07-05 08:57:41 (465 KB/s) - ‘linux_input.txt’ saved [6206996/6206996]



In [1]:
import codecs
import random
import numpy as np

class CharModel:
    
    def __init__(self, file_path, nlen, reverse):
        self._nlen = nlen
        self._ngrams = dict()
        self._vocabs = dict()
        
        last_chars = ['\\']*nlen
       
        with codecs.open(file_path, "r", "ISO-8859-1") as f:
            text = f.read()
            if reverse:
                text = ''.join(reversed(text))
                
            for ch in text:
                if ch not in self._vocabs:
                    self._vocabs[ch] = 0
                self._vocabs[ch] += 1

                query = ''.join(last_chars);
                if query not in self._ngrams:
                     self._ngrams[query] = dict();
                if ch not in self._ngrams[query]:
                    self._ngrams[query][ch] = 0;
                self._ngrams[query][ch] += 1;
                del last_chars[0:1]
                last_chars.append(ch);
        for query, chars in self._ngrams.items():
            sm = float(sum(chars.values()))
            for ch in chars:
                chars[ch] /= sm
    def next_letter(self, chars):
        stats = self.stats(chars);
        c_keys = list(stats.keys())
        c_p = list(stats.values())
        c_next = np.random.choice(c_keys, 1, p=c_p)[0];
        return c_next

    def stats(self, chars):
        if chars in self._ngrams:
            return self._ngrams[chars]
        return dict()
    def gen_text(self, text_len):
        last_chars = ['\\'] * self._nlen
        res = []
        for i in range(text_len):
            ch = self.next_letter(''.join(last_chars))
            del last_chars[0:1]
            last_chars.append(ch);
            
            res.append(ch)
        return "".join(res)

deep_len = 30
char_model = CharModel('linux_input.txt', deep_len, False)
char_model_rev = CharModel('linux_input.txt', deep_len, True)
#print(char_model._ngrams)
#print(char_model.stats('pri'))
#for _ in range(10):
    #print(char_model.next_letter('pri'))\
need_len = 10000;
last_text =  char_model_rev.gen_text(deep_len) 
text = char_model.gen_text(need_len - deep_len) + ''.join(reversed(last_text))
print(text)


/*
 * linux/kernel/irq/autoprobe.c
 *
 * Copyright (C) 1992, 1998-2006 Linus Torvalds, Ingo Molnar
 * Copyright (C) 2005-2006, Thomas Gleixner <tglx@linutronix.de>
 *  Copyright (C) 2008-2011 Red Hat, Inc., Ingo Molnar
 *  Copyright (C) 2004 Nadia Yvette Chambers
 */
#include <linux/ring_buffer.h>
#include <linux/debugfs.h>
#include <linux/tracefs.h>
#include <linux/types.h>
#include <linux/slab.h>
#include <linux/sched.h>
#include <linux/sched/rt.h>
#include <linux/export.h>

unsigned int __read_mostly softlockup_panic =
			CONFIG_BOOTPARAM_HARDLOCKUP_PANIC_VALUE;
/*
 * We may not want to enable hard lockup detection by default in all cases,
 * for example when running the kernel as a guest on a hypervisor. In these
 * cases this function can be called to disable hard lockup detection. This
 * function should only be executed once by the boot processor before the
 * kernel command line parameters passed at boot.
 *
 *  Scheduler profiling support, Arjan van de Ven and Ingo Molnar,
 *	